# Coursera - IBM Applied Data Science Capstone Project

### Good place to open new  Shopping mall in Bangalore, India

- Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall


###  Import Libraries

In [1]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML
from IPython.display import display_html

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

##### Creating a dataframe of Bangalore Neighbourhoods

In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_7e3fbfd4137643858e9cc630a3240c2e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tMMrJxHIj4OIKrDU_L4KvL_NuD5kmquPn6_Piq0LDpkr',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_7e3fbfd4137643858e9cc630a3240c2e.get_object(Bucket='capstonefinalproject-donotdelete-pr-uy5yb3uqw3u8wl',Key='Bangalore-Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data = pd.read_csv(body,delimiter = '\t')
df_data.head()


,Borough,Neighborhoods,Latitude,Longitude,Population,City,AverageIncome
0,Central,Cantonment area,12.972442,77.580643,866377,Bangalore,18944.099792
1,Central,Domlur,12.960992,77.638726,743186,Bangalore,56837.022198
2,Central,Indiranagar,12.971891,77.641151,474289,Bangalore,41991.817435
3,Central,Jeevanbheemanagar,12.962900,77.659500,527874,Bangalore,6667.447632
4,Central,Malleswaram,13.003100,77.564300,893629,Bangalore,53270.063892


In [3]:
df = df_data[['Neighborhoods ','Latitude ','Longitude ']]
df.rename(columns={'Neighborhoods ': 'Neighborhood'},inplace=True)
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Neighborhood,Latitude,Longitude
0,Cantonment area,12.972442,77.580643
1,Domlur,12.960992,77.638726
2,Indiranagar,12.971891,77.641151
3,Jeevanbheemanagar,12.962900,77.659500
4,Malleswaram,13.003100,77.564300


In [4]:
# get the coordinates of Bangalore
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [5]:
# create map of Bangalore using latitude and longitude values
map_bl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude '], df['Longitude '], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bl)
map_bl
    

### Use FourSquare API to explore the Neighbourhoods

In [8]:
# @hidden_cell
CLIENT_ID = 'VCLJ1QFQZMYIBW1AAODHEGAGI4LV1GW4UFONNCVFFWGLKSFS' # your Foursquare ID
CLIENT_SECRET = 'YN0TXBMKSSB3JG4ZZ2LFS1Y1D5SWHVYU5HJ3ZSVVETA4DQWT' # your Foursquare Secret
VERSION = '20180605'

In [9]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude '], df['Longitude '], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [10]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3457, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cantonment area,12.972442,77.580643,Hotel Fishland,12.975569,77.578592,Seafood Restaurant
1,Cantonment area,12.972442,77.580643,Chikkana Tiffin Room,12.968557,77.584282,Indian Restaurant
2,Cantonment area,12.972442,77.580643,Lakshmi Nataraj Refreshments,12.967476,77.579468,South Indian Restaurant
3,Cantonment area,12.972442,77.580643,Taj West End,12.984572,77.584893,Hotel
4,Cantonment area,12.972442,77.580643,JW Marriott Hotel Bengaluru,12.972362,77.595051,Hotel


In [11]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Anjanapura,2,2,2,2,2,2
Arekere,89,89,89,89,89,89
BTM Layout,100,100,100,100,100,100
Banaswadi,62,62,62,62,62,62
Basavanagudi,100,100,100,100,100,100
Basaveshwaranagar,65,65,65,65,65,65
Begur,9,9,9,9,9,9
Bellandur,95,95,95,95,95,95
Bommanahalli,37,37,37,37,37,37


#### Let's find out how many unique categories can be curated from all the returned venues

In [12]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 212 uniques categories.


In [13]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Seafood Restaurant', 'Indian Restaurant',
       'South Indian Restaurant', 'Hotel', 'Theater', 'Racetrack',
       'Shopping Mall', 'Italian Restaurant', 'Park', 'Lounge',
       'Asian Restaurant', 'French Restaurant', 'Japanese Restaurant',
       'Ice Cream Shop', 'Bakery', 'Cupcake Shop', 'Deli / Bodega',
       'Fried Chicken Joint', 'Clothing Store', 'Bookstore',
       'Furniture / Home Store', 'Dessert Shop', 'Art Gallery',
       'Cocktail Bar', 'Brewery', 'Hotel Bar', 'Coffee Shop',
       'Vegetarian / Vegan Restaurant', 'Golf Course',
       'Vietnamese Restaurant', 'Capitol Building', 'Boutique', 'Arcade',
       'Wine Bar', 'Karnataka Restaurant', 'Restaurant',
       'Bed & Breakfast', 'Mexican Restaurant', 'Pub', 'Track Stadium',
       'Chinese Restaurant', 'Spa', 'Parsi Restaurant',
       'Sushi Restaurant', 'Breakfast Spot', 'Bagel Shop',
       'Motorcycle Shop', 'Café', 'Movie Theater', 'Stadium'],
      dtype=object)

In [14]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### Analyse Each Neighborhood

In [15]:
# one hot encoding
bl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bl_onehot.columns[-1]] + list(bl_onehot.columns[:-1])
bl_onehot = bl_onehot[fixed_columns]

print(bl_onehot.shape)
bl_onehot.head()

(3457, 213)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
bl_grouped = bl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(bl_grouped.shape)
bl_grouped

(63, 213)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Anjanapura,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Arekere,0.000000,0.000000,0.011236,0.011236,0.000000,0.000000,0.000000,0.000000,0.022472,...,0.000000,0.000000,0.00,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,BTM Layout,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Banaswadi,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.016129,0.000000,...,0.000000,0.000000,0.00,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Basavanagudi,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.010000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00
5,Basaveshwaranagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,...,0.000000,0.000000,0.00,0.046154,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Begur,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Bellandur,0.000000,0.010526,0.010526,0.010526,0.000000,0.000000,0.000000,0.000000,0.010526,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Bommanahalli,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.027027,...,0.000000,0.000000,0.00,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Bommasandra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [17]:
len(bl_grouped[bl_grouped["Shopping Mall"] > 0])

30

#### Create a new DataFrame for Shopping Mall data only

In [18]:
bl_mall = bl_grouped[["Neighborhoods","Shopping Mall"]]

In [19]:
bl_mall.head()

,Neighborhoods,Shopping Mall
0,Anjanapura,0.000000
1,Arekere,0.022472
2,BTM Layout,0.010000
3,Banaswadi,0.016129
4,Basavanagudi,0.000000


### Cluster the neighbourhoods

In [20]:
# set number of clusters
kclusters = 3

kl_clustering = bl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



array([1, 0, 1, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [21]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bl_merged = bl_mall.copy()

# add clustering labels
bl_merged["Cluster Labels"] = kmeans.labels_

In [22]:
bl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Anjanapura,0.000000,1
1,Arekere,0.022472,0
2,BTM Layout,0.010000,1
3,Banaswadi,0.016129,0
4,Basavanagudi,0.000000,1


In [23]:
# merge Bangalore_grouped with Bangalore_data to add latitude/longitude for each neighborhood
bl_merged = bl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(bl_merged.shape)
bl_merged.head() # check the last columns!

(63, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,1,12.8604,77.5612
1,Arekere,0.022472,0,12.8875,77.5970
2,BTM Layout,0.010000,1,12.9166,77.6101
3,Banaswadi,0.016129,0,13.0120,77.6471
4,Basavanagudi,0.000000,1,12.9421,77.5754


In [24]:
# sort the results by Cluster Labels
print(bl_merged.shape)
bl_merged.sort_values(["Cluster Labels"], inplace=True)
bl_merged

(63, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
31,Kothnur,0.028571,0,12.873400,77.582000
32,Krishnarajapuram,0.035088,0,13.004000,77.687800
38,Malleswaram,0.020000,0,13.003100,77.564300
28,Kammanahalli,0.029412,0,13.015900,77.637900
42,Nandini Layout,0.038462,0,13.016000,77.533800
23,J. P. Nagar,0.020000,0,12.910500,77.585700
44,Padmanabhanagar,0.028571,0,12.915600,77.556800
21,Hulimavu,0.016667,0,12.878900,77.609000
19,Hoodi,0.039216,0,12.992200,77.715900
36,Mahadevapura,0.027027,0,12.991300,77.687400


#### Finally let's Visualize the resulting clusters

In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bl_merged['Latitude '], bl_merged['Longitude '], bl_merged['Neighborhood'], bl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
bl_merged.loc[bl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
31,Kothnur,0.028571,0,12.8734,77.5820
32,Krishnarajapuram,0.035088,0,13.0040,77.6878
38,Malleswaram,0.020000,0,13.0031,77.5643
28,Kammanahalli,0.029412,0,13.0159,77.6379
42,Nandini Layout,0.038462,0,13.0160,77.5338
23,J. P. Nagar,0.020000,0,12.9105,77.5857
44,Padmanabhanagar,0.028571,0,12.9156,77.5568
21,Hulimavu,0.016667,0,12.8789,77.6090
19,Hoodi,0.039216,0,12.9922,77.7159
36,Mahadevapura,0.027027,0,12.9913,77.6874


#### Cluster1

In [27]:
bl_merged.loc[bl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
52,Seshadripuram,0.010000,1,12.993500,77.578700
37,Mahalakshmi Layout,0.013333,1,13.014600,77.551400
39,Marathahalli,0.000000,1,12.959200,77.697400
40,Mathikere,0.012500,1,13.033400,77.564000
41,Nagarbhavi,0.000000,1,12.959900,77.508300
59,Vijayanagar,0.000000,1,12.971900,77.529900
58,Vidyaranyapura,0.000000,1,13.085800,77.556100
57,Vasanth Nagar,0.000000,1,12.991100,77.592000
56,Varthur,0.000000,1,12.938900,77.741200
46,Pete area,0.012500,1,12.962700,77.575800


#### Cluster 2

In [28]:
bl_merged.loc[bl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
45,Peenya,0.083333,2,13.0285,77.5197


### Observations

Most of the shopping malls are concentrated in the Eastern part of Bangalore city, with the highest number in cluster 0 and moderate number in cluster 1. On the other hand, cluster 2 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of shopping malls and suffering from intense competition.
